In [349]:
import pandas as pd
import numpy as np
from mlxtend.plotting import plot_decision_regions

In [350]:
df = pd.DataFrame()

In [351]:
df['X1'] = [1,2,3,4,5,61,7,8,9,3]
df['X2'] = [3,5,1,7,6,3,121,2,3,4]
df['label'] = [1,0,1,1,0,1,0,0,1,1]

In [352]:
df

,X1,X2,label
0,1,3,1
1,2,5,0
2,3,1,1
3,4,7,1
4,5,6,0
5,61,3,1
6,7,121,0
7,8,2,0
8,9,3,1
9,3,4,1


In [353]:
dt_array=[]
alpha_array= []
df_array=[]
df_array.append(df)

In [354]:
def find_error(df):
    return 1-np.sum(df.loc[df['label']==df['y_pred'],'weights'])

In [355]:

def calculate_alpha(error):
    if error==0:
        return 0
    else:
        return 0.5*np.log((1-error)/error)


In [356]:
def update_row_weights(row,alpha):
    if row['label']==row['y_pred']:
        return row['weights']*np.exp(-alpha)
    else:
        return row['weights']*np.exp(alpha)
    

In [357]:
def find_index(df):
    indices=[]
    for i in range(df.shape[0]):
        num = np.random.rand()
        for index,row in df.iterrows():
            if(row['lower']<=num and row['upper']>num):
                indices.append(index)
                break
    return indices


In [358]:
def adaBoost(df):
    df['weights'] = 1/df.shape[0]
    dt = DecisionTreeClassifier(max_depth=1)
    dt_array.append(dt)
    X = df.iloc[:,0:2]
    y = df.iloc[:,2]
    dt.fit(X,y)
    df['y_pred'] = dt.predict(X)
    error = find_error(df)
    alpha_array.append(calculate_alpha(error))
    df['updated_weights']=df.apply(update_row_weights,alpha=alpha_array[-1],axis=1)
    df['normalised_weight']=df['updated_weights']/df['updated_weights'].sum()
    df['upper']=np.cumsum(df['normalised_weight'])
    df['lower'] = df['upper']-df['normalised_weight']
    index_array = find_index(df)
    df = pd.DataFrame()
    print(df)
    print(df_array[-1].iloc[index_array,[0,1,2,3]])
    return df_array[-1].iloc[index_array,[0,1,2,3]]


In [359]:
for i in range(5):
    df_array.append(adaBoost(df_array[i]))

Empty DataFrame
Columns: []
Index: []
   X1  X2  label  weights
7   8   2      0      0.1
3   4   7      1      0.1
4   5   6      0      0.1
2   3   1      1      0.1
4   5   6      0      0.1
5  61   3      1      0.1
7   8   2      0      0.1
3   4   7      1      0.1
2   3   1      1      0.1
5  61   3      1      0.1
Empty DataFrame
Columns: []
Index: []
   X1  X2  label  weights
4   5   6      0      0.1
5  61   3      1      0.1
4   5   6      0      0.1
4   5   6      0      0.1
2   3   1      1      0.1
4   5   6      0      0.1
4   5   6      0      0.1
2   3   1      1      0.1
5  61   3      1      0.1
3   4   7      1      0.1
Empty DataFrame
Columns: []
Index: []
   X1  X2  label  weights
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
4   5   6      0      0.1
Empty DataFrame
Columns: []


In [360]:
df[['X1','X2','label','weights','y_pred','updated_weights','lower','upper']]

,X1,X2,label,weights,y_pred,updated_weights,lower,upper
0,1,3,1,0.1,1,0.05,0.0000,0.0625
1,2,5,0,0.1,0,0.05,0.0625,0.1250
2,3,1,1,0.1,1,0.05,0.1250,0.1875
3,4,7,1,0.1,0,0.20,0.1875,0.4375
4,5,6,0,0.1,0,0.05,0.4375,0.5000
5,61,3,1,0.1,1,0.05,0.5000,0.5625
6,7,121,0,0.1,0,0.05,0.5625,0.6250
7,8,2,0,0.1,1,0.20,0.6250,0.8750
8,9,3,1,0.1,1,0.05,0.8750,0.9375
9,3,4,1,0.1,1,0.05,0.9375,1.0000


In [361]:
print(df_array.__len__())

6


In [362]:
dt_array

[DecisionTreeClassifier(max_depth=1),
 DecisionTreeClassifier(max_depth=1),
 DecisionTreeClassifier(max_depth=1),
 DecisionTreeClassifier(max_depth=1),
 DecisionTreeClassifier(max_depth=1)]

In [363]:
alpha_array

[0.6931471805599454, 0.6931471805599454, 1.0986122886681098, 0, 0]

In [364]:
def result(query):
    ans = 0
    for i in range(4):
        print(np.sign(ans))
        ans =  ans +  alpha_array[i]*dt_array[i].predict(query)[0]
        
    return np.sign(ans)

In [367]:
query = np.array([7,121]).reshape(1,2)
result(query)

0
0.0
0.0
0.0


/Users/spydee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/spydee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/spydee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/spydee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.0